## 1. Import Libraries

# Bitcoin Price Prediction Using Machine Learning
Nhập môn Học máy và Khai phá dữ liệu (IT3190)

In [ ]:
# Data manipulation and analysis
import numpy as np
import pandas as pd
import math

# Machine learning
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV

# ML Algorithms
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.svm import SVR, SVC
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier

# Deep learning
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
# Thêm vào phần import
import tensorflow as tf
from tensorflow import keras
# Time series
import datetime as dt

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

## 2. Data Loading and Exploration

In [ ]:
# Tải dữ liệu
print("Đang tải dữ liệu lịch sử Bitcoin...")

# Đường dẫn đến tệp dữ liệu
duong_dan = '/kaggle/input/bitcoin-historical-data/btcusd_1-min_data.csv'

# Đọc dữ liệu từ tệp CSV
df = pd.read_csv(duong_dan)

# Hiển thị thông tin cơ bản
print("\nThông tin về tập dữ liệu:")
print(f"Kích thước tập dữ liệu: {df.shape}")

print("\nKiểu dữ liệu của các cột:")
print(df.dtypes)


In [ ]:
# Chuyển đổi cột timestamp sang dạng datetime
df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='s')

# Đặt timestamp làm chỉ mục
df.set_index('Timestamp', inplace=True)

# Hiển thị 5 dòng đầu tiên của tập dữ liệu
print("\n5 dòng đầu tiên của tập dữ liệu:")
print(df.head())


In [ ]:
# Thống kê cơ bản
print("\nThống kê mô tả của tập dữ liệu:")
print(df.describe())


In [ ]:
# Kiểm tra giá trị bị thiếu
print("\nGiá trị bị thiếu trong tập dữ liệu:")
print(df.isnull().sum())

# Loại bỏ các giá trị bị thiếu
df.dropna(inplace=True)
print(f"\nKích thước tập dữ liệu sau khi loại bỏ giá trị bị thiếu: {df.shape}")


In [ ]:
# Check for duplicates
duplicates = df.duplicated().sum()
print(f"\nSố lượng dòng trùng lặp: {duplicates}")

## 3. Data Visualization

In [ ]:
# Vẽ biểu đồ giá Bitcoin theo thời gian
plt.figure(figsize=(15, 6))
plt.plot(df['Close'], label='Giá đóng cửa của Bitcoin (USD)')
plt.title('Biến động giá Bitcoin theo thời gian')
plt.xlabel('Ngày')
plt.ylabel('Giá (USD)')
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# Lấy mẫu lại dữ liệu để vẽ biểu đồ theo các khoảng thời gian khác nhau
plt.figure(figsize=(15, 12))

plt.subplot(4, 1, 1)
plt.plot(df['Close'].resample('D').mean(), label='Trung bình ngày')
plt.title('Giá Bitcoin trung bình theo ngày')
plt.legend()

plt.subplot(4, 1, 2)
plt.plot(df['Close'].resample('W').mean(), label='Trung bình tuần')
plt.title('Giá Bitcoin trung bình theo tuần')
plt.legend()

plt.subplot(4, 1, 3)
plt.plot(df['Close'].resample('M').mean(), label='Trung bình tháng')
plt.title('Giá Bitcoin trung bình theo tháng')
plt.legend()

plt.subplot(4, 1, 4)
plt.plot(df['Volume'].resample('W').sum(), label='Khối lượng giao dịch tuần')
plt.title('Khối lượng giao dịch Bitcoin theo tuần')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
# Tạo bản đồ nhiệt hiển thị mối tương quan
plt.figure(figsize=(10, 8))
correlation = df.corr()
sns.heatmap(correlation, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Bản đồ nhiệt của ma trận tương quan')
plt.tight_layout()
plt.show()


In [ ]:
# Biểu đồ tần suất lợi nhuận hàng ngày
df_daily = df.resample('D').last()
df_daily['Returns'] = df_daily['Close'].pct_change() * 100

plt.figure(figsize=(12, 6))
plt.hist(df_daily['Returns'].dropna(), bins=50, alpha=0.75)
plt.title('Phân bố lợi nhuận hàng ngày')
plt.xlabel('Lợi nhuận hàng ngày (%)')
plt.ylabel('Tần suất')
plt.axvline(0, color='red', linestyle='--')
plt.tight_layout()
plt.show()


## 4. Data Preprocessing

In [ ]:
# Lấy mẫu lại dữ liệu theo ngày với phương pháp tổng hợp phù hợp
df_daily = df.resample('D').agg({
    'Open': 'first',
    'High': 'max',
    'Low': 'min',
    'Close': 'last',
    'Volume': 'sum'
}).dropna()

print(f"\nKích thước dữ liệu sau khi tổng hợp theo ngày: {df_daily.shape}")


In [ ]:
# Kỹ thuật tạo đặc trưng (Feature Engineering)
def enhanced_technical_indicators(df):
    """
    Hàm nâng cao để thêm các chỉ báo kỹ thuật vào DataFrame
    """
    # Các chỉ báo kỹ thuật từ phiên bản cũ
    # Đường trung bình động (Moving Averages)
    df['MA7'] = df['Close'].rolling(window=7).mean()
    df['MA14'] = df['Close'].rolling(window=14).mean()
    df['MA30'] = df['Close'].rolling(window=30).mean()
    df['MA20'] = df['Close'].rolling(window=20).mean()  # Thêm MA20 cho Bollinger Bands
    
    # Động lượng giá (Price Momentum)
    df['Price_Momentum'] = df['Close'] - df['Close'].shift(7)
    
    # Độ biến động (Volatility - độ lệch chuẩn của lợi nhuận)
    df['Volatility'] = df['Close'].pct_change().rolling(window=7).std() * 100
    
    # Tỷ lệ thay đổi giá (Rate of Change - ROC)
    df['ROC'] = ((df['Close'] - df['Close'].shift(7)) / df['Close'].shift(7)) * 100
    
    # Chỉ số sức mạnh tương đối (Relative Strength Index - RSI)
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    
    # Đường trung bình hội tụ phân kỳ (MACD)
    ema12 = df['Close'].ewm(span=12, adjust=False).mean()
    ema26 = df['Close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = ema12 - ema26
    df['MACD_Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
    
    # THÊM CÁC CHỈ BÁO MỚI DƯỚI ĐÂY:
    
    # Dải Bollinger (Bollinger Bands)
    std_dev = df['Close'].rolling(window=20).std()
    df['BB_Upper'] = df['MA20'] + (std_dev * 2)
    df['BB_Lower'] = df['MA20'] - (std_dev * 2)
    df['BB_Width'] = (df['BB_Upper'] - df['BB_Lower']) / df['MA20']
    df['BB_Pct'] = (df['Close'] - df['BB_Lower']) / (df['BB_Upper'] - df['BB_Lower'])
    
    # Chỉ báo độ biến động - Average True Range (ATR)
    high_low = df['High'] - df['Low']
    high_close = np.abs(df['High'] - df['Close'].shift())
    low_close = np.abs(df['Low'] - df['Close'].shift())
    true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    df['ATR'] = true_range.rolling(14).mean()
    
    # Chỉ báo khối lượng - On-Balance Volume (OBV)
    df['OBV'] = 0
    df.loc[df.index[1:], 'OBV'] = (df['Volume'][1:] * 
                                  ((df['Close'][1:] > df['Close'].shift(1)[1:]).astype(int) - 
                                   (df['Close'][1:] < df['Close'].shift(1)[1:]).astype(int))).cumsum()
    
    # Dao động Stochastic (Stochastic Oscillator)
    n_period = 14
    df['n_high'] = df['High'].rolling(n_period).max()
    df['n_low'] = df['Low'].rolling(n_period).min()
    # Tính %K (fast stochastic)
    df['%K'] = ((df['Close'] - df['n_low']) / (df['n_high'] - df['n_low'])) * 100
    # Tính %D (slow stochastic) - trung bình động 3 kỳ của %K
    df['%D'] = df['%K'].rolling(3).mean()
    
    # Chỉ số kênh hàng hóa (Commodity Channel Index - CCI)
    typical_price = (df['High'] + df['Low'] + df['Close']) / 3
    mean_dev = np.abs(typical_price - typical_price.rolling(window=20).mean()).rolling(window=20).mean()
    df['CCI'] = (typical_price - typical_price.rolling(window=20).mean()) / (0.015 * mean_dev)
    
    # Các thành phần của mây Ichimoku (Ichimoku Cloud)
    high_9 = df['High'].rolling(window=9).max()
    low_9 = df['Low'].rolling(window=9).min()
    df['Tenkan_sen'] = (high_9 + low_9) / 2  # Đường chuyển (Conversion Line)
    
    high_26 = df['High'].rolling(window=26).max()
    low_26 = df['Low'].rolling(window=26).min()
    df['Kijun_sen'] = (high_26 + low_26) / 2  # Đường nền (Base Line)
    
    # Chỉ số dòng tiền (Money Flow Index - MFI)
    typical_price = (df['High'] + df['Low'] + df['Close']) / 3
    raw_money_flow = typical_price * df['Volume']
    
    # Tạo chuỗi dòng tiền dương và âm
    df['money_flow'] = raw_money_flow
    df['money_flow_pos'] = np.where(typical_price > typical_price.shift(1), df['money_flow'], 0)
    df['money_flow_neg'] = np.where(typical_price < typical_price.shift(1), df['money_flow'], 0)
    
    # Tính tỷ lệ dòng tiền
    df['money_flow_pos_sum'] = df['money_flow_pos'].rolling(window=14).sum()
    df['money_flow_neg_sum'] = df['money_flow_neg'].rolling(window=14).sum()
    
    # Tính chỉ số dòng tiền
    money_ratio = df['money_flow_pos_sum'] / df['money_flow_neg_sum']
    df['MFI'] = 100 - (100 / (1 + money_ratio))
    
    # Tỷ lệ thay đổi giá (Price Rate of Change) với các chu kỳ khác nhau
    df['ROC_5'] = ((df['Close'] - df['Close'].shift(5)) / df['Close'].shift(5)) * 100
    df['ROC_10'] = ((df['Close'] - df['Close'].shift(10)) / df['Close'].shift(10)) * 100
    df['ROC_15'] = ((df['Close'] - df['Close'].shift(15)) / df['Close'].shift(15)) * 100
    
    # Độ biến động lịch sử (Historical Volatility) với các chu kỳ khác nhau
    df['HV_10'] = df['Close'].pct_change().rolling(window=10).std() * np.sqrt(252) * 100
    df['HV_20'] = df['Close'].pct_change().rolling(window=20).std() * np.sqrt(252) * 100
    
    # Dao động khối lượng phần trăm (Percentage Volume Oscillator - PVO)
    volume_12_ema = df['Volume'].ewm(span=12, adjust=False).mean()
    volume_26_ema = df['Volume'].ewm(span=26, adjust=False).mean()
    df['PVO'] = ((volume_12_ema - volume_26_ema) / volume_26_ema) * 100
    
    # Xu hướng giá và khối lượng (Price and Volume Trend)
    df['PVT'] = 0
    df.loc[df.index[1:], 'PVT'] = (
        (df['Close'][1:] - df['Close'].shift(1)[1:]) / df['Close'].shift(1)[1:] * df['Volume'][1:]
    ).cumsum()
    
    # Tín hiệu cắt nhau của đường trung bình động đơn giản (SMA Cross-overs)
    df['SMA_Cross_5_20'] = (df['Close'].rolling(window=5).mean() > 
                           df['Close'].rolling(window=20).mean()).astype(int)
    
    # Biến mục tiêu cho mô hình phân loại (giữ nguyên từ bản gốc)
    df['Target_Next_Day'] = np.where(df['Close'].shift(-1) > df['Close'], 1, 0)
    df['Target_Next_Week'] = np.where(df['Close'].shift(-7) > df['Close'], 1, 0)
    
    # Loại bỏ các cột tạm thời
    drop_cols = ['n_high', 'n_low', 'money_flow', 'money_flow_pos', 'money_flow_neg', 
                 'money_flow_pos_sum', 'money_flow_neg_sum']
    df = df.drop(drop_cols, axis=1, errors='ignore')
    
    return df

# Thêm các chỉ báo kỹ thuật
# df_daily = add_technical_indicators(df_daily)  # Comment out or remove this line
df_daily = enhanced_technical_indicators(df_daily)  # Replace with enhanced version

# Loại bỏ các giá trị NaN sau khi thêm chỉ báo kỹ thuật
df_daily.dropna(inplace=True)
print(f"\nKích thước dữ liệu sau khi thêm chỉ báo kỹ thuật và loại bỏ NaN: {df_daily.shape}")


In [ ]:
# Hiển thị tập dữ liệu đã được mở rộng
print("\nXem trước tập dữ liệu sau khi bổ sung chỉ báo kỹ thuật:")
print(df_daily.head())


In [ ]:
# Trực quan hóa một số chỉ báo kỹ thuật
plt.figure(figsize=(15, 10))

plt.subplot(3, 1, 1)
plt.plot(df_daily['Close'], label='Giá đóng cửa')
plt.plot(df_daily['MA7'], label='Đường trung bình 7 ngày')
plt.plot(df_daily['MA30'], label='Đường trung bình 30 ngày')
plt.title('Giá Bitcoin với đường trung bình động')
plt.legend()

plt.subplot(3, 1, 2)
plt.plot(df_daily['RSI'], label='RSI')
plt.axhline(y=70, color='r', linestyle='--')
plt.axhline(y=30, color='g', linestyle='--')
plt.title('Chỉ số sức mạnh tương đối (RSI)')
plt.legend()

plt.subplot(3, 1, 3)
plt.plot(df_daily['MACD'], label='MACD')
plt.plot(df_daily['MACD_Signal'], label='Đường tín hiệu')
plt.title('Đường trung bình hội tụ phân kỳ (MACD)')
plt.legend()

plt.tight_layout()
plt.show()
# Trực quan hóa một số chỉ báo kỹ thuật mới
plt.figure(figsize=(15, 15))

# Dải Bollinger
plt.subplot(5, 1, 1)
plt.plot(df_daily['Close'], label='Giá đóng cửa')
plt.plot(df_daily['BB_Upper'], label='BB Upper', color='red', alpha=0.7)
plt.plot(df_daily['MA20'], label='MA20', color='orange', alpha=0.7)
plt.plot(df_daily['BB_Lower'], label='BB Lower', color='green', alpha=0.7)
plt.title('Dải Bollinger')
plt.legend()

# ATR - Biên độ dao động thực
plt.subplot(5, 1, 2)
plt.plot(df_daily['ATR'], label='ATR', color='purple')
plt.title('Biên độ dao động thực (ATR)')
plt.legend()

# Dao động Stochastic
plt.subplot(5, 1, 3)
plt.plot(df_daily['%K'], label='%K', color='blue')
plt.plot(df_daily['%D'], label='%D', color='red')
plt.axhline(y=80, color='red', linestyle='--', alpha=0.5)
plt.axhline(y=20, color='green', linestyle='--', alpha=0.5)
plt.title('Dao động Stochastic')
plt.legend()

# Khối lượng tích lũy (OBV)
plt.subplot(5, 1, 4)
plt.plot(df_daily['OBV'], label='OBV', color='darkblue')
plt.title('Khối lượng tích lũy (OBV)')
plt.legend()

# Chỉ số dòng tiền (MFI)
plt.subplot(5, 1, 5)
plt.plot(df_daily['MFI'], label='MFI', color='green')
plt.axhline(y=80, color='red', linestyle='--', alpha=0.5)
plt.axhline(y=20, color='green', linestyle='--', alpha=0.5)
plt.title('Chỉ số dòng tiền (MFI)')
plt.legend()

plt.tight_layout()
plt.show()

## 5. Preparing Data for Machine Learning

In [ ]:
# Hàm tạo chuỗi dữ liệu cho dự báo chuỗi thời gian
def create_sequences(data, target_col, window_size):
    """Tạo chuỗi dữ liệu cho mô hình dự báo chuỗi thời gian"""
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data.iloc[i:i+window_size].values)
        y.append(data.iloc[i+window_size][target_col])
    return np.array(X), np.array(y)

# Hàm chuẩn bị dữ liệu cho mô hình hồi quy/phân loại
def prepare_data_for_ml(df, target_col, window_size=0, is_classification=False):
    """Chuẩn bị dữ liệu cho các mô hình máy học"""
    # Dành cho mô hình chuỗi thời gian (sử dụng window_size)
    if window_size > 0:
        X, y = create_sequences(df, target_col, window_size)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
    
    # Dành cho các mô hình truyền thống
    else:
        # Chọn đặc trưng và biến mục tiêu
        X = df.drop(['Open', 'High', 'Low', 'Close', 'Target_Next_Day', 'Target_Next_Week'], axis=1) 
        if is_classification:
            y = df[target_col].astype(int)
        else:
            y = df['Close']
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    return X_train, X_test, y_train, y_test


In [ ]:
# Tách đặc trưng và biến mục tiêu cho mô hình hồi quy
X = df_daily.drop(['Open', 'High', 'Low', 'Close', 'Target_Next_Day', 'Target_Next_Week'], axis=1)
y_reg = df_daily['Close']

# Chuẩn hóa dữ liệu cho mô hình hồi quy
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled = scaler_X.fit_transform(X)
y_reg_scaled = scaler_y.fit_transform(y_reg.values.reshape(-1, 1)).flatten()

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_reg_scaled, test_size=0.2, random_state=42)

print(f"\nKích thước tập huấn luyện: {X_train.shape}")
print(f"Kích thước tập kiểm tra: {X_test.shape}")


In [ ]:
# Chuẩn bị dữ liệu cho mô hình phân loại
X_class = df_daily.drop(['Open', 'High', 'Low', 'Close', 'Target_Next_Day', 'Target_Next_Week'], axis=1)
y_class = df_daily['Target_Next_Day']

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra cho mô hình phân loại
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(X_scaled, y_class, test_size=0.2, random_state=42)


In [ ]:
# Dữ liệu cho mô hình chuỗi thời gian (LSTM)
time_steps = 30  # Số ngày quan sát trước đó
X_ts, y_ts = create_sequences(df_daily[['Close'] + list(X.columns)], 'Close', time_steps)

# Chuẩn hóa dữ liệu chuỗi thời gian
scaler_ts = MinMaxScaler()
# CORRECT APPROACH - Reshape, scale once, then reshape back
X_ts_reshaped = X_ts.reshape(-1, X_ts.shape[2])
X_ts_scaled = scaler_ts.fit_transform(X_ts_reshaped)
X_ts_scaled = X_ts_scaled.reshape(X_ts.shape)
y_ts_scaled = scaler_y.transform(y_ts.reshape(-1, 1)).flatten()

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
split_idx = int(0.8 * len(X_ts_scaled))
X_train_ts, X_test_ts = X_ts_scaled[:split_idx], X_ts_scaled[split_idx:]
y_train_ts, y_test_ts = y_ts_scaled[:split_idx], y_ts_scaled[split_idx:]

print(f"\nKích thước tập huấn luyện chuỗi thời gian: {X_train_ts.shape}")
print(f"Kích thước tập kiểm tra chuỗi thời gian: {X_test_ts.shape}")


## 6. Model Development and Training

### 6.1 Linear Regression

In [ ]:
# Huấn luyện mô hình hồi quy tuyến tính
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Dự đoán giá trị
y_pred_lr = lr_model.predict(X_test)

# Đánh giá mô hình
lr_rmse = np.sqrt(mean_squared_error(y_test, y_pred_lr))
lr_mae = mean_absolute_error(y_test, y_pred_lr)
lr_r2 = r2_score(y_test, y_pred_lr)

print("\n=== Kết quả Hồi quy tuyến tính ===")
print(f"RMSE: {lr_rmse:.4f}")
print(f"MAE: {lr_mae:.4f}")
print(f"R^2: {lr_r2:.4f}")


### A. Regression Models

In [ ]:
def train_and_evaluate_regression_models(X_train, X_test, y_train, y_test):
    """Huấn luyện và đánh giá nhiều mô hình hồi quy"""
    results = {}
    
    # Hồi quy cây quyết định (Decision Tree Regressor)
    dt_model = DecisionTreeRegressor(random_state=42)
    dt_model.fit(X_train, y_train)
    y_pred_dt = dt_model.predict(X_test)
    results['Decision Tree'] = {
        'model': dt_model,
        'rmse': np.sqrt(mean_squared_error(y_test, y_pred_dt)),
        'mae': mean_absolute_error(y_test, y_pred_dt),
        'r2': r2_score(y_test, y_pred_dt)
    }
    
    # Hồi quy rừng ngẫu nhiên (Random Forest Regressor)
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
    rf_model.fit(X_train, y_train)
    y_pred_rf = rf_model.predict(X_test)
    results['Random Forest'] = {
        'model': rf_model,
        'rmse': np.sqrt(mean_squared_error(y_test, y_pred_rf)),
        'mae': mean_absolute_error(y_test, y_pred_rf),
        'r2': r2_score(y_test, y_pred_rf)
    }
    
    # Hồi quy vector hỗ trợ (Support Vector Regression - SVR)
    svr_model = SVR(kernel='rbf')
    svr_model.fit(X_train, y_train)
    y_pred_svr = svr_model.predict(X_test)
    results['SVR'] = {
        'model': svr_model,
        'rmse': np.sqrt(mean_squared_error(y_test, y_pred_svr)),
        'mae': mean_absolute_error(y_test, y_pred_svr),
        'r2': r2_score(y_test, y_pred_svr)
    }
    
    # Hồi quy K-Nearest Neighbors (KNN Regressor)
    knn_model = KNeighborsRegressor(n_neighbors=5)
    knn_model.fit(X_train, y_train)
    y_pred_knn = knn_model.predict(X_test)
    results['KNN'] = {
        'model': knn_model,
        'rmse': np.sqrt(mean_squared_error(y_test, y_pred_knn)),
        'mae': mean_absolute_error(y_test, y_pred_knn),
        'r2': r2_score(y_test, y_pred_knn)
    }
    
    return results

# Huấn luyện và đánh giá các mô hình hồi quy
regression_results = train_and_evaluate_regression_models(X_train, X_test, y_train, y_test)

# Hiển thị kết quả của các mô hình hồi quy
print("\n=== Kết quả các mô hình hồi quy ===")
for model_name, result in regression_results.items():
    print(f"\nKết quả {model_name}:")
    print(f"RMSE: {result['rmse']:.4f}")
    print(f"MAE: {result['mae']:.4f}")
    print(f"R^2: {result['r2']:.4f}")


### B. Classification Models

In [ ]:
def train_and_evaluate_classification_models(X_train, X_test, y_train, y_test):
    """Huấn luyện và đánh giá nhiều mô hình phân loại"""
    results = {}

    # Cây quyết định (Decision Tree Classifier)
    dt_model = DecisionTreeClassifier(random_state=42)
    dt_model.fit(X_train, y_train)
    y_pred_dt = dt_model.predict(X_test)
    results['Cây quyết định'] = {
        'model': dt_model,
        'độ chính xác': accuracy_score(y_test, y_pred_dt),
        'độ chính xác (precision)': precision_score(y_test, y_pred_dt),
        'độ nhạy (recall)': recall_score(y_test, y_pred_dt),
        'F1 Score': f1_score(y_test, y_pred_dt)
    }

    # Rừng ngẫu nhiên (Random Forest Classifier)
    rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_model.fit(X_train, y_train)
    y_pred_rf = rf_model.predict(X_test)
    results['Rừng ngẫu nhiên'] = {
        'model': rf_model,
        'độ chính xác': accuracy_score(y_test, y_pred_rf),
        'độ chính xác (precision)': precision_score(y_test, y_pred_rf),
        'độ nhạy (recall)': recall_score(y_test, y_pred_rf),
        'F1 Score': f1_score(y_test, y_pred_rf)
    }

    # Máy vector hỗ trợ (Support Vector Machine Classifier)
    svc_model = SVC(kernel='rbf', probability=True)
    svc_model.fit(X_train, y_train)
    y_pred_svc = svc_model.predict(X_test)
    results['SVC'] = {
        'model': svc_model,
        'độ chính xác': accuracy_score(y_test, y_pred_svc),
        'độ chính xác (precision)': precision_score(y_test, y_pred_svc),
        'độ nhạy (recall)': recall_score(y_test, y_pred_svc),
        'F1 Score': f1_score(y_test, y_pred_svc)
    }

    # K-láng giềng gần nhất (K-Nearest Neighbors Classifier)
    knn_model = KNeighborsClassifier(n_neighbors=5)
    knn_model.fit(X_train, y_train)
    y_pred_knn = knn_model.predict(X_test)
    results['KNN'] = {
        'model': knn_model,
        'độ chính xác': accuracy_score(y_test, y_pred_knn),
        'độ chính xác (precision)': precision_score(y_test, y_pred_knn),
        'độ nhạy (recall)': recall_score(y_test, y_pred_knn),
        'F1 Score': f1_score(y_test, y_pred_knn)
    }

    return results

# Huấn luyện và đánh giá các mô hình phân loại
classification_results = train_and_evaluate_classification_models(X_train_class, X_test_class, y_train_class, y_test_class)

# Hiển thị kết quả phân loại
print("\n=== Kết quả mô hình phân loại (Dự đoán xu hướng giá) ===")
for model_name, result in classification_results.items():
    print(f"\nKết quả của {model_name}:")
    print(f"Độ chính xác: {result['độ chính xác']:.4f}")
    print(f"Độ chính xác (Precision): {result['độ chính xác (precision)']:.4f}")
    print(f"Độ nhạy (Recall): {result['độ nhạy (recall)']:.4f}")
    print(f"F1 Score: {result['F1 Score']:.4f}")


### C. Time Series Models (LSTM)

In [ ]:
# Định dạng lại dữ liệu đầu vào cho LSTM [mẫu, bước thời gian, đặc trưng]
n_features = X_train_ts.shape[2]
X_train_ts_reshaped = X_train_ts.reshape(X_train_ts.shape[0], time_steps, n_features)
X_test_ts_reshaped = X_test_ts.reshape(X_test_ts.shape[0], time_steps, n_features)

# Enhanced LSTM architecture with more capacity
def create_lstm_model(input_shape):
    model = Sequential()
    # First LSTM layer with more units
    model.add(LSTM(units=128, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.3))
    # Second LSTM layer
    model.add(LSTM(units=64, return_sequences=True))
    model.add(Dropout(0.3))
    # Third LSTM layer
    model.add(LSTM(units=64, return_sequences=False))
    model.add(Dropout(0.3))
    # Dense layers with batch normalization
    model.add(Dense(units=32, activation='relu'))
    model.add(Dense(units=16, activation='relu'))
    model.add(Dense(units=1))
    
    # Use a more sophisticated optimizer
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='huber')  # Huber loss is more robust to outliers
    return model

# Tạo và huấn luyện mô hình LSTM
lstm_model = create_lstm_model((time_steps, n_features))
print("\nTóm tắt mô hình LSTM:")
lstm_model.summary()


In [ ]:
# Định nghĩa các callback cho quá trình huấn luyện
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_lstm_model.keras', monitor='val_loss', save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)

# Huấn luyện mô hình LSTM
lstm_history = lstm_model.fit(
    X_train_ts_reshaped, y_train_ts,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping, checkpoint, reduce_lr],
    verbose=1
)


In [ ]:
# Đánh giá mô hình LSTM
lstm_test_loss = lstm_model.evaluate(X_test_ts_reshaped, y_test_ts, verbose=0)
lstm_predictions = lstm_model.predict(X_test_ts_reshaped)

# Chuyển đổi dự đoán về thang đo ban đầu
lstm_predictions = scaler_y.inverse_transform(lstm_predictions)
y_test_original = scaler_y.inverse_transform(y_test_ts.reshape(-1, 1))

# Tính toán các chỉ số đánh giá hiệu suất
lstm_rmse = np.sqrt(mean_squared_error(y_test_original, lstm_predictions))
lstm_mae = mean_absolute_error(y_test_original, lstm_predictions)
lstm_r2 = r2_score(y_test_original, lstm_predictions)

print("\n=== Kết quả mô hình LSTM ===")
print(f"Độ mất mát trên tập kiểm tra: {lstm_test_loss:.4f}")
print(f"RMSE: {lstm_rmse:.4f}")
print(f"MAE: {lstm_mae:.4f}")
print(f"R^2: {lstm_r2:.4f}")


## 7. Model Evaluation and Comparison

In [ ]:
# Vẽ biểu đồ lịch sử huấn luyện của mô hình LSTM
plt.figure(figsize=(12, 6))
plt.plot(lstm_history.history['loss'], label='Mất mát trên tập huấn luyện')
plt.plot(lstm_history.history['val_loss'], label='Mất mát trên tập kiểm định')
plt.title('Lịch sử huấn luyện mô hình LSTM')
plt.xlabel('Số epoch')
plt.ylabel('Mất mát (MSE)')
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# Vẽ biểu đồ so sánh giữa giá thực tế và dự đoán của mô hình LSTM
plt.figure(figsize=(15, 6))
plt.plot(y_test_original, label='Giá Bitcoin thực tế', color='blue')
plt.plot(lstm_predictions, label='Dự đoán của LSTM', color='red', alpha=0.7)
plt.title('Dự đoán giá Bitcoin: Thực tế vs Dự đoán LSTM')
plt.xlabel('Thời gian')
plt.ylabel('Giá (USD)')
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# Lấy mô hình hồi quy có hiệu suất tốt nhất
best_reg_model_name = min(regression_results, key=lambda k: regression_results[k]['rmse'])
best_reg_model = regression_results[best_reg_model_name]['model']

# Lấy tầm quan trọng của đặc trưng nếu mô hình tốt nhất là Random Forest
if best_reg_model_name == 'Random Forest':
    feature_importances = best_reg_model.feature_importances_
    feature_names = X.columns
    
    # Tạo DataFrame chứa tầm quan trọng của đặc trưng và sắp xếp
    importance_df = pd.DataFrame({
        'Đặc trưng': feature_names,
        'Tầm quan trọng': feature_importances
    })
    importance_df = importance_df.sort_values('Tầm quan trọng', ascending=False)
    
    # Vẽ biểu đồ tầm quan trọng của đặc trưng
    plt.figure(figsize=(12, 6))
    sns.barplot(x='Tầm quan trọng', y='Đặc trưng', data=importance_df)
    plt.title('Tầm quan trọng của đặc trưng trong dự đoán giá Bitcoin')
    plt.tight_layout()
    plt.show()


In [ ]:
# So sánh hiệu suất của tất cả các mô hình hồi quy
plt.figure(figsize=(12, 6))
models = list(regression_results.keys()) + ['LSTM']
rmse_values = [regression_results[model]['rmse'] for model in regression_results] + [lstm_rmse]
colors = ['blue', 'green', 'red', 'purple', 'orange']  # Màu sắc cho từng mô hình

plt.bar(models, rmse_values, color=colors)
plt.title('So sánh RMSE giữa các mô hình')
plt.xlabel('Mô hình')
plt.ylabel('RMSE')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
# So sánh hiệu suất của các mô hình phân loại
plt.figure(figsize=(12, 6))
classifiers = list(classification_results.keys())
metrics = ['độ chính xác', 'độ chính xác (precision)', 'độ nhạy (recall)', 'F1 Score']
metric_values = {
    metric: [classification_results[model][metric] for model in classifiers]
    for metric in metrics
}

x = np.arange(len(classifiers))  # Tạo chỉ số cho trục x
width = 0.2  # Độ rộng của mỗi cột
multiplier = 0  # Hệ số dịch chuyển

fig, ax = plt.subplots(figsize=(12, 6))

# Duyệt qua từng metric và vẽ biểu đồ cột
for metric, values in metric_values.items():
    offset = width * multiplier
    # Use a simplified label for the legend
    label = metric.split(' ')[0] if '(' in metric else metric
    ax.bar(x + offset, values, width, label=label)
    multiplier += 1

ax.set_title('So sánh các chỉ số hiệu suất mô hình phân loại')
ax.set_xticks(x + width * (len(metrics) - 1) / 2)
ax.set_xticklabels(classifiers)
ax.set_ylim(0, 1)
ax.set_ylabel('Điểm số')
ax.legend(loc='lower right')

plt.tight_layout()
plt.show()

## 8. Model Optimization with Cross-Validation

In [ ]:
# Định nghĩa hàm thực hiện k-fold cross-validation
def perform_kfold_cv(model, X, y, k=5):
    """Thực hiện k-fold cross-validation"""
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    rmse_scores = []
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        rmse_scores.append(rmse)
    
    return rmse_scores

# Thực hiện cross-validation cho mô hình hồi quy tốt nhất
if best_reg_model_name == 'Random Forest':
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
    rf_cv_scores = perform_kfold_cv(rf_model, X_scaled, y_reg_scaled)
    
    print(f"\nKết quả 5-Fold CV của Random Forest:")
    print(f"RMSE trên các lần chạy: {rf_cv_scores}")
    print(f"RMSE trung bình: {np.mean(rf_cv_scores):.4f}")
    print(f"Độ lệch chuẩn RMSE: {np.std(rf_cv_scores):.4f}")
    
    # Tinh chỉnh siêu tham số bằng GridSearchCV
    param_grid = {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }
    
    grid_search = GridSearchCV(RandomForestRegressor(random_state=42), param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    grid_search.fit(X_scaled, y_reg_scaled)
    
    print("\nSiêu tham số tối ưu cho Random Forest:")
    print(grid_search.best_params_)
    
    # Huấn luyện mô hình tối ưu
    best_rf_model = grid_search.best_estimator_
    best_rf_model.fit(X_train, y_train)
    y_pred_best_rf = best_rf_model.predict(X_test)
    
    # Đánh giá mô hình tối ưu
    best_rf_rmse = np.sqrt(mean_squared_error(y_test, y_pred_best_rf))
    best_rf_mae = mean_absolute_error(y_test, y_pred_best_rf)
    best_rf_r2 = r2_score(y_test, y_pred_best_rf)
    
    print("\nKết quả mô hình Random Forest tối ưu:")
    print(f"RMSE: {best_rf_rmse:.4f}")
    print(f"MAE: {best_rf_mae:.4f}")
    print(f"R^2: {best_rf_r2:.4f}")


## 9. Predicting Future Bitcoin Prices

In [ ]:
# Sử dụng mô hình tốt nhất để dự đoán giá tương lai
best_model = best_rf_model if best_reg_model_name == 'Random Forest' else best_reg_model

# Hàm dự đoán giá cho ngày tiếp theo
def predict_next_day_price(model, last_data, scaler_x, scaler_y):
    """Dự đoán giá Bitcoin cho ngày tiếp theo sử dụng mô hình đã huấn luyện"""
    # Chuẩn hóa dữ liệu đầu vào
    last_data_scaled = scaler_x.transform(last_data.reshape(1, -1))
    
    # Dự đoán giá
    prediction_scaled = model.predict(last_data_scaled)
    
    # Chuyển đổi về thang đo thực tế
    prediction = scaler_y.inverse_transform(prediction_scaled.reshape(-1, 1))
    
    return prediction[0][0]

# Lấy điểm dữ liệu cuối cùng có sẵn
last_data = X_scaled[-1]

# Dự đoán giá cho ngày tiếp theo
next_day_price = predict_next_day_price(best_model, last_data, scaler_X, scaler_y)
print(f"\nDự đoán giá Bitcoin cho ngày tiếp theo: ${next_day_price:.2f}")


In [ ]:
# Dự đoán giá tương lai với mô hình LSTM
def predict_next_day_lstm(model, last_sequence, scaler):
    """Dự đoán giá Bitcoin cho ngày tiếp theo sử dụng mô hình LSTM"""
    # Định dạng lại dữ liệu đầu vào cho LSTM
    last_sequence_reshaped = last_sequence.reshape(1, time_steps, n_features)
    
    # Thực hiện dự đoán
    prediction_scaled = model.predict(last_sequence_reshaped)
    
    # Chuyển đổi về thang đo thực tế
    prediction = scaler.inverse_transform(prediction_scaled)
    
    return prediction[0][0]

# Lấy chuỗi dữ liệu cuối cùng để dự đoán với LSTM
last_sequence = X_test_ts_reshaped[-1]

# Dự đoán giá cho ngày tiếp theo bằng LSTM
next_day_price_lstm = predict_next_day_lstm(lstm_model, last_sequence, scaler_y)
print(f"Dự đoán giá Bitcoin cho ngày tiếp theo (LSTM): ${next_day_price_lstm:.2f}")


## 10. Multi-day Price Forecasting

In [ ]:
# Hàm dự đoán giá Bitcoin cho nhiều ngày tiếp theo
def improved_forecast_future_prices(model, last_data, last_prices, feature_names, days=7, is_lstm=False):
    """Dự đoán giá Bitcoin trong một số ngày tới với cập nhật đặc trưng"""
    forecasted_prices = []
    current_data = last_data.copy()
    
    # Lấy giá cuối cùng từ dữ liệu thực tế
    last_price = last_prices[-1]
    
    # Danh sách lưu trữ các giá dự đoán để tính đặc trưng
    price_history = list(last_prices)
    
    for day in range(days):
        if is_lstm:
            # Xử lý cho LSTM (không thay đổi)
            current_data_reshaped = current_data.reshape(1, time_steps, n_features)
            next_price = model.predict(current_data_reshaped)[0][0]
            current_data = np.roll(current_data, -1, axis=0)
            current_data[-1, 0] = next_price
        else:
            # Dự đoán giá tiếp theo dựa trên dữ liệu hiện tại
            next_price = model.predict(current_data.reshape(1, -1))[0]
            
            # Thêm giá dự đoán vào lịch sử giá
            price_history.append(next_price)
            
            # Cập nhật các đặc trưng kỹ thuật dựa trên giá mới dự đoán
            # Ví dụ: Cập nhật MA7 (cần phải biết vị trí của MA7 trong vector đặc trưng)
            if 'MA7' in feature_names:
                ma7_idx = feature_names.index('MA7')
                current_data[0, ma7_idx] = np.mean(price_history[-7:])
            
            # Cập nhật MA14
            if 'MA14' in feature_names:
                ma14_idx = feature_names.index('MA14')
                current_data[0, ma14_idx] = np.mean(price_history[-14:])
            
            # Cập nhật ROC (Rate of Change)
            if 'ROC' in feature_names:
                roc_idx = feature_names.index('ROC')
                if len(price_history) > 7:
                    current_data[0, roc_idx] = ((price_history[-1] - price_history[-8]) / price_history[-8]) * 100
            
            # Tương tự cho các đặc trưng khác...
        
        forecasted_prices.append(next_price)
    
    return forecasted_prices
# Lựa chọn mô hình tốt nhất dựa trên đánh giá
if lstm_rmse < best_rf_rmse:
    best_overall_model = lstm_model
    last_data_for_forecast = last_sequence
    is_lstm = True
    print("\nLSTM là mô hình có hiệu suất tốt nhất.")
else:
    best_overall_model = best_model
    last_data_for_forecast = last_data
    is_lstm = False
    print(f"\n{best_reg_model_name} là mô hình có hiệu suất tốt nhất.")

# Dự đoán giá cho 7 ngày tiếp theo
forecast_days = 7

# Lấy tên các đặc trưng
feature_names = X.columns.tolist()

# Lấy dữ liệu giá gần đây để tính toán các chỉ báo
recent_prices = df_daily['Close'].values[-30:]  # Lấy 30 giá gần nhất

# Sử dụng hàm cải tiến
future_prices = improved_forecast_future_prices(
    best_overall_model, 
    last_data_for_forecast, 
    recent_prices, 
    feature_names,
    forecast_days, 
    is_lstm
)


# Chuyển đổi dữ liệu về giá trị thực nếu mô hình không phải LSTM
if not is_lstm:
    future_prices = scaler_y.inverse_transform(np.array(future_prices).reshape(-1, 1)).flatten()

# Tạo dãy ngày cho dự báo
last_date = df_daily.index[-1]
future_dates = [last_date + dt.timedelta(days=i+1) for i in range(forecast_days)]

# Hiển thị kết quả dự báo
print(f"\nDự đoán giá Bitcoin trong {forecast_days} ngày tới:")
for date, price in zip(future_dates, future_prices):
    print(f"{date.date()}: ${price:.2f}")


In [ ]:
# Vẽ biểu đồ kết quả chiến lược giao dịch
plt.figure(figsize=(12, 6))

# Xác định best_class_model nếu chưa được định nghĩa
if 'best_class_model' not in globals():
    # Lấy mô hình phân loại tốt nhất
    best_class_model_name = max(classification_results, key=lambda k: classification_results[k]['độ chính xác'])
    best_class_model = classification_results[best_class_model_name]['model']
    
    print(f"Sử dụng mô hình phân loại tốt nhất: {best_class_model_name}")
    print(f"Độ chính xác: {classification_results[best_class_model_name]['độ chính xác']:.4f}")

# Lấy giá thực tế từ dữ liệu kiểm tra
actual_prices = scaler_y.inverse_transform(y_test.reshape(-1, 1)).flatten()

# Dự đoán xu hướng giá
y_pred = best_class_model.predict(X_test_class)

# Vẽ giá thực tế
plt.plot(actual_prices, label="Giá thực tế", color='blue', alpha=0.6)

# Đánh dấu các điểm mua
buy_signals = [i for i in range(len(y_pred)) if y_pred[i] == 1]
buy_prices = [actual_prices[i] for i in buy_signals]
plt.scatter(buy_signals, buy_prices, marker="^", color="green", label="Mua", alpha=1)

# Đánh dấu các điểm bán
sell_signals = [i for i in range(len(y_pred)) if y_pred[i] == 0]
sell_prices = [actual_prices[i] for i in sell_signals]
plt.scatter(sell_signals, sell_prices, marker="v", color="red", label="Bán", alpha=1)

plt.title("Chiến lược giao dịch dựa trên mô hình phân loại")
plt.xlabel("Thời gian")
plt.ylabel("Giá Bitcoin (USD)")
plt.legend()
plt.tight_layout()
plt.show()

## 11. Trading Strategy Based on Price Direction Prediction

In [ ]:
# Lấy mô hình phân loại tốt nhất
best_class_model_name = max(classification_results, key=lambda k: classification_results[k]['độ chính xác'])
best_class_model = classification_results[best_class_model_name]['model']

print(f"\nMô hình phân loại tốt nhất: {best_class_model_name}")
print(f"Độ chính xác: {classification_results[best_class_model_name]['độ chính xác']:.4f}")

def backtest_trading_strategy(model, X_test, y_test, vốn_ban_đầu=10000):
    vốn = vốn_ban_đầu
    lượng_btc = 0
    nhật_ký_giao_dịch = []
    
    # Dự đoán
    y_pred = model.predict(X_test)
    
    # Lấy giá thực tế
    giá_thực_tế = scaler_y.inverse_transform(y_test.reshape(-1, 1)).flatten()
    
    # Constants for realistic constraints
    fee_rate = 0.002  # 0.2% fee (more realistic)
    max_position_size = 0.2  # Max 20% of capital per trade
    max_btc_per_trade = 10  # Maximum BTC per single trade (realistic limit)
    slippage = 0.005  # 0.5% slippage on large orders
    
    for i in range(len(y_pred) - 1):
        giá_hiện_tại = giá_thực_tế[i]
        
        # Skip trades if price is too low (prevents division by zero and unrealistic scenarios)
        if giá_hiện_tại <= 0.01:
            continue
            
        if y_pred[i] == 1 and lượng_btc == 0 and vốn > 100:  # Buy signal with minimum capital
            # Calculate affordable BTC with position sizing
            usable_capital = min(vốn * max_position_size, vốn - 100)  # Keep min $100 reserve
            
            # Apply slippage to buy price for realism
            adjusted_price = giá_hiện_tại * (1 + slippage)
            
            # Calculate BTC to buy with fees
            btc_mua = (usable_capital * (1 - fee_rate)) / adjusted_price
            
            # Apply maximum BTC limit per trade
            btc_mua = min(btc_mua, max_btc_per_trade)
            
            # Execute trade
            trade_cost = btc_mua * adjusted_price * (1 + fee_rate)
            
            if trade_cost <= vốn:  # Ensure we have enough capital
                lượng_btc = btc_mua
                vốn -= trade_cost
                nhật_ký_giao_dịch.append(f"Ngày {i}: Mua {btc_mua:.6f} BTC với giá ${adjusted_price:.2f}, Chi phí: ${trade_cost:.2f}")
            
        elif y_pred[i] == 0 and lượng_btc > 0:  # Sell signal
            # Apply slippage to sell price
            adjusted_price = giá_hiện_tại * (1 - slippage)
            
            # Calculate proceeds with fees
            vốn_thu = lượng_btc * adjusted_price * (1 - fee_rate)
            
            nhật_ký_giao_dịch.append(f"Ngày {i}: Bán {lượng_btc:.6f} BTC với giá ${adjusted_price:.2f}, Thu: ${vốn_thu:.2f}, Vốn: ${(vốn + vốn_thu):.2f}")
            vốn += vốn_thu
            lượng_btc = 0
    
    # Final evaluation
    if lượng_btc > 0:
        adjusted_final_price = giá_thực_tế[-1] * (1 - slippage)
        vốn_cuối = vốn + (lượng_btc * adjusted_final_price * (1 - fee_rate))
    else:
        vốn_cuối = vốn
    
    lợi_nhuận = vốn_cuối - vốn_ban_đầu
    phần_trăm_lợi_nhuận = (lợi_nhuận / vốn_ban_đầu) * 100
    
    return {
        'vốn_ban_đầu': vốn_ban_đầu,
        'vốn_cuối': vốn_cuối,
        'lợi_nhuận': lợi_nhuận,
        'phần_trăm_lợi_nhuận': phần_trăm_lợi_nhuận,
        'nhật_ký_giao_dịch': nhật_ký_giao_dịch,
        'số_giao_dịch': len(nhật_ký_giao_dịch)
    }

# Chạy backtest cho chiến lược giao dịch
kết_quả_chiến_lược = backtest_trading_strategy(best_class_model, X_test_class, y_test, 10000)

print("\n=== Kết quả Backtest Chiến lược Giao dịch ===")
print(f"Vốn ban đầu: ${kết_quả_chiến_lược['vốn_ban_đầu']:.2f}")
print(f"Vốn cuối: ${kết_quả_chiến_lược['vốn_cuối']:.2f}")
print(f"Lợi nhuận: ${kết_quả_chiến_lược['lợi_nhuận']:.2f} ({kết_quả_chiến_lược['phần_trăm_lợi_nhuận']:.2f}%)")

print("\nNhật ký giao dịch (5 giao dịch đầu và 5 giao dịch cuối):")
for log in kết_quả_chiến_lược['nhật_ký_giao_dịch'][:5]:
    print(log)
print("...")
for log in kết_quả_chiến_lược['nhật_ký_giao_dịch'][-5:]:
    print(log)


## 12. Conclusion and Future Improvements

In [ ]:
print("\\n=== Kết luận ===")
print("Trong dự án này, chúng tôi đã xây dựng và đánh giá thành công nhiều mô hình học máy để dự đoán giá Bitcoin.")

# Hiển thị mô hình tốt nhất cho các nhiệm vụ khác nhau
print("\\nMô hình tốt nhất cho từng nhiệm vụ dự đoán:")
print(f"1. Hồi quy giá: {best_reg_model_name if lstm_rmse > best_rf_rmse else 'LSTM'}")
print(f"2. Phân loại xu hướng giá: {best_class_model_name}")

print("\\nNhững phát hiện chính:")
print("1. Các chỉ báo kỹ thuật cung cấp đặc trưng giá trị trong dự đoán giá Bitcoin")
print("2. Mô hình chuỗi thời gian như LSTM có thể nắm bắt sự phụ thuộc theo thời gian của giá Bitcoin")
print("3. Tính biến động cao của thị trường tiền mã hóa khiến việc dự đoán chính xác trở nên khó khăn")

# Check if kết_quả_chiến_lược exists, otherwise use a default message
if 'kết_quả_chiến_lược' in globals():
    print(f"4. Chiến lược giao dịch của chúng tôi dựa trên {best_class_model_name} cho thấy kết quả {'đầy hứa hẹn' if kết_quả_chiến_lược['lợi_nhuận'] > 0 else 'hạn chế'}")
else:
    print(f"4. Chiến lược giao dịch của chúng tôi dựa trên mô hình phân loại cần được đánh giá kỹ lưỡng hơn")

print("\\nCải tiến trong tương lai:")
print("1. Tích hợp phân tích cảm xúc từ mạng xã hội và tin tức")
print("2. Bổ sung các chỉ số kinh tế vĩ mô và dữ liệu thị trường")
print("3. Thử nghiệm với các kiến trúc học sâu tiên tiến hơn")
print("4. Phát triển các chiến lược giao dịch tinh vi hơn")
print("5. Triển khai học trực tuyến để thích ứng với điều kiện thị trường thay đổi")

In [ ]:
# Biểu đồ cuối cùng hiển thị dự đoán so với giá thực tế
plt.figure(figsize=(15, 8))

# Tạo dãy ngày để trực quan hóa dữ liệu kiểm tra
ngày_kiểm_tra = df_daily.index[-len(y_test):]

# Vẽ giá thực tế
giá_thực_tế = scaler_y.inverse_transform(y_test.reshape(-1, 1))
plt.plot(ngày_kiểm_tra, giá_thực_tế, label='Giá thực tế', color='blue')

# Fix plot dimensions
plt.plot(ngày_kiểm_tra[:len(lstm_predictions)], lstm_predictions, label='Dự đoán LSTM', color='red', alpha=0.7)

# Vẽ dự đoán từ mô hình hồi quy tốt nhất
dự_đoán_hồi_quy = best_model.predict(X_test)
dự_đoán_hồi_quy = scaler_y.inverse_transform(dự_đoán_hồi_quy.reshape(-1, 1))
plt.plot(ngày_kiểm_tra, dự_đoán_hồi_quy[:len(ngày_kiểm_tra)], label=f'Dự đoán {best_reg_model_name}', color='green', alpha=0.7)

# Vẽ giá dự báo trong tương lai
plt.plot(future_dates, future_prices, 'o-', label='Dự báo tương lai', color='purple')

plt.title('Tóm tắt dự đoán giá Bitcoin')
plt.xlabel('Ngày')
plt.ylabel('Giá (USD)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

print("\nCảm ơn bạn đã cùng chúng tôi khám phá dự đoán giá Bitcoin!")


In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression

# Prepare the feature data
X_for_selection = df_daily.drop(['Open', 'High', 'Low', 'Close', 'Target_Next_Day', 'Target_Next_Week'], axis=1)
y_for_selection = df_daily['Close']

# Apply feature selection
selector = SelectKBest(score_func=f_regression, k=15)  # Select top 15 features
X_new = selector.fit_transform(X_for_selection, y_for_selection)

# Get selected feature names
feature_scores = pd.DataFrame({
    'Feature': X_for_selection.columns,
    'Score': selector.scores_
})
top_features = feature_scores.sort_values(by='Score', ascending=False).head(15)

print("Top 15 most important features:")
print(top_features)

# Plot feature importance
plt.figure(figsize=(12, 8))
sns.barplot(x='Score', y='Feature', data=top_features)
plt.title('Top 15 Most Important Features')
plt.tight_layout()
plt.show()

# Update your feature selection
# You might want to use only these top features in your models
important_features = top_features['Feature'].tolist()
print("Important features to use in models:", important_features)